In [8]:
import csv  
import re

with open('Sentiment140_data/training.1600000.processed.noemoticon.csv', mode='r', encoding = "ISO-8859-1") as csv_file:  
    csv_reader = csv.DictReader(csv_file, fieldnames=['target', 'id', 'date', 'flag', 'user', 'text'])
    line = 0
    with open('tweets.valid', "w", encoding="utf-8") as f1:
        with open('tweets.train', "w", encoding="utf-8") as f2:
            for row in csv_reader:
                # Clean the training data
                # First we lower case the text
                text = row["text"].lower()
                # remove links
                text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
                #Remove usernames
                text = re.sub('@[^\s]+','', text)
                # replace hashtags by just words
                text = re.sub(r'#([^\s]+)', r'\1', text)
                #correct all multiple white spaces to a single white space
                text = re.sub('[\s]+', ' ', text)
                # Additional clean up : removing words less than 3 chars, and remove space at the beginning and the end
                text = re.sub(r'\W*\b\w{1,3}\b', '', text)

                text = re.sub('\x80', '', text)

                text = text.strip()
                line = line + 1
                # Split data into train and validation
                if line%20 == 0:
                    f1.write(f'__label__{row["target"]} {text}')
                    f1.write('\n')
                else:
                    f2.write(f'__label__{row["target"]} {text}')
                    f2.write('\n')

In [1]:
import pandas as pd  
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("Sentiment140_data/training.1600000.processed.noemoticon.csv", header=None, \
                 encoding = "ISO-8859-1", names=cols)
df.drop(['id','date','query_string','user'],axis=1,inplace=True)
clean_tweet_texts = []
for i in range(len(df['text'])):
    clean_tweet_texts.append(tweet_cleaner_updated(df['text'][i]))
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['target'] = df.sentiment
clean_df.to_csv('clean_tweet.csv',encoding='utf-8')

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b' i just received my G8 viola exam.. and its... well... .. disappointing.. :\\..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'E3 ON PLAYSTATION HOME IN ABOUT AN HOUR!!!!!!!!!! \\../  \\../'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [2]:
import json
import re

with open('clean_data.json', "w", encoding="utf-8") as f1:
    for line in open('data.json', 'r'):
        dic = json.loads(line)
        new_dic = {}
        new_dic['repo_name'] = dic['repo_name']
        new_dic["created_at"] = dic["created_at"]
        new_dic["closed_at"] = dic["closed_at"]
        text = dic["body"]
        if text:
            text = text.lower()

            text = text.encode('ascii', 'ignore').decode("utf-8")
            text = re.sub('<[^>]+>','', text)
            text = re.sub('[\n]+', ' ', text)
            text = re.sub('[\r]+', ' ', text)
            text = re.sub(r'````[^````]+````','', text)
            text = re.sub(r'```[^```]+```','', text)
            text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
            text = re.sub('@[^\s]+','', text)
            text = re.sub(r'#([^\s]+)', r'\1', text)
            text = re.sub('[\s]+', ' ', text)
            text = re.sub(r'\W*\b\w{1,3}\b', '', text)
            text = re.sub('\x80', '', text)
            text = text.strip()
            
            new_dic["body"] = text
            json.dump(new_dic, f1)
            f1.write('\n')